<a href="https://colab.research.google.com/github/team5INFO5731/INFO5731-FinalProject/blob/main/hmm_lda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
!pip install BeautifulSoup
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

In [ ]:
!pip install beautifulsoup4

In [ ]:
## short, news data

def get_html_loadmore(url,num_pages):
  import requests
  from bs4 import BeautifulSoup
  import pandas as pd
  from selenium import webdriver
  import time
  from selenium.webdriver.support.ui import WebDriverWait
  from selenium.webdriver.common.by import By
  from selenium.webdriver.support import expected_conditions as EC
  from selenium import webdriver
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

  url = url
  wd.get(url)
  html = wd.page_source.encode('utf-8')
  page_num = 0
  while (wd.find_elements_by_id('load-more-btn') and page_num<num_pages):
    wd.find_element_by_id('load-more-btn').click()
    time.sleep(3)
    page_num+=1
  html = wd.page_source.encode('utf-8')
  return html
  



In [ ]:
from bs4 import BeautifulSoup
short_text=[]
news=[]
html =get_html_loadmore("https://inshorts.com/en/read",40)
soup = BeautifulSoup(html, "lxml")
short_text_soup=soup.find_all('div', class_=["news-card-title news-right-box"])
news_soup=soup.find_all('div', class_=["news-card-content news-right-box"])

for i in short_text_soup:
  short_text.append(i.find('span',attrs={'itemprop':"headline"}).string)

for j in news_soup:
  news.append(j.find('div',attrs={'itemprop':"articleBody"}).string)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: use options instead of chrome_options


In [ ]:
import pandas as pd
short_text_df=(pd.DataFrame(short_text)).to_csv("short_text.csv")
news_text_df=(pd.DataFrame(news)).to_csv("news.csv")

In [ ]:
## social media tweet data
import tweepy
consumer_key = "v6DkFAg34mL5bnQa6HgUycOoZ"
consumer_secret = "KNqZqEa2FWx0cnLZ7aLwqo01Q12oYSxeb3khK21lbNibULQAEn"
access_token = "3992308705-oEgS72kYOAPPr7mLBVmDTXWzKgvIe50VEwJNm6P"
access_token_secret = "o1lILiDx3MhFO47Es91Xjt2zxwPH7S04Hml2nvqbfb8i0"
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

In [ ]:
pip install tweepy

In [ ]:
text_query = 'Joe biden'
count = 1000
try:
 # Creation of query method using parameters
 tweets = tweepy.Cursor(api.search,q=text_query).items(count)
 
 # Pulling information from tweets iterable object
 tweets_list = [[tweet.created_at, tweet.id, tweet.text] for tweet in tweets]
 
 # Creation of dataframe from tweets list
 # Add or remove columns as you remove tweet information
 social_media = pd.DataFrame(tweets_list)
 
except BaseException as e:
    print('failed on_status,',str(e))
    time.sleep(3)
social_media.to_csv("social_media.csv")

In [ ]:
#scientifc_articles

import pandas as pd
import requests
from bs4 import BeautifulSoup
abstracts=[]
def insert_into_array(values,ref_array):
  for i in values:
    ref_array.append(i.text)
titles=[]
for i in range(0,5000,10):
  URL="http://citeseerx.ist.psu.edu/search?q=lda&t=doc&sort=rlv&start="+str(i)
  page=requests.get(URL,headers={'User-Agent':'Chrome/85.0.4183.121'})
  soup = BeautifulSoup(page.content, 'html.parser')
  insert_into_array(soup.find_all('a', class_='remove doc_details'),titles)
  insert_into_array(soup.find_all('div', class_='snippet'),abstracts)


In [ ]:
scientific_df = pd.DataFrame({"titles":titles,"abstarcts":abstracts})
scientific_df.to_csv("scientific_articles.csv")

# Text cleaning( removing punctuations, stop words, lower casing, stemming, lemmatization)

In [ ]:
def clean_text(listt):
  from nltk.stem import PorterStemmer
  import pandas as pd
  import nltk
  nltk.download('punkt')
  from nltk.corpus import stopwords
  nltk.download('stopwords')
  from textblob import Word
  nltk.download('wordnet')
  stop = stopwords.words('english')
  imdb_df=pd.DataFrame()
  imdb_df['comment']=listt
  imdb_df['after_punct_less'] = imdb_df['comment'].str.replace('[^\w\s].#','') 
  imdb_df['after_rm_stopwords'] =imdb_df['after_punct_less'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
  imdb_df['after_numerics']=imdb_df['after_rm_stopwords'].str.replace('[0-9]','')
  imdb_df['after_lowercasing'] =imdb_df['after_numerics'].apply(lambda x: " ".join(x.lower() for x in x.split()))
  from nltk.stem import PorterStemmer
  st = PorterStemmer()
  #imdb_df['after_stemming']=imdb_df['after_lowercasing'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))
  #imdb_df['after_lemmatization'] = imdb_df['after_stemming'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
  return (imdb_df['after_lowercasing']).tolist()

# HMM_LDA

In [ ]:
!git clone "https://github.com/dongwookim-ml/python-topic-model.git"

Cloning into 'python-topic-model'...
remote: Enumerating objects: 357, done.
remote: Total 357 (delta 0), reused 0 (delta 0), pack-reused 357
Receiving objects: 100% (357/357), 18.75 MiB | 12.76 MiB/s, done.
Resolving deltas: 100% (181/181), done.


In [ ]:
import sys
sys.path.insert(0, '/content/python-topic-model')

In [ ]:
import os
import logging

import numpy as np
import matplotlib.pyplot as plt

from ptm import GibbsSupervisedLDA
from ptm.nltk_corpus import get_ids_cnt
from ptm.utils import convert_cnt_to_list, get_top_words
from ptm.nltk_corpus import get_reuters_token_list_by_sentence
from ptm import HMM_LDA
from ptm.utils import get_top_words

logger = logging.getLogger('HMM_LDA')
logger.propagate=False

%matplotlib inline  

logger = logging.getLogger('GibbsSupervisedLDA')
logger.propagate = False

#short text

In [ ]:
import nltk
nltk.download('words')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
def hmm_lda(documents):
  voca, word_ids, word_cnt = get_ids_cnt(documents)
  corpus = convert_cnt_to_list(word_ids, word_cnt)
  corpus=[corpus]
  n_docs = len(corpus)
  n_voca = len(voca)
  n_topic = 20
  n_class = 15
  max_iter = 100
  alpha = 0.1
  beta = 0.01
  gamma = 0.1
  eta = 0.1
  model = HMM_LDA(n_docs, n_voca, n_topic, n_class, alpha=alpha, beta=beta, gamma=gamma, eta=eta, verbose=False)
  model.fit(corpus, max_iter=max_iter)
  for ti in range(n_topic):
    top_words = get_top_words(model.TW, voca, ti, n_words=10)
    print('Topic', ti ,': ', ','.join(top_words))

In [ ]:
short_text[:10]

In [ ]:
# HMM LDA for short text:
cleaned_short_text=clean_text(short_text)
hmm_lda(cleaned_short_text)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Topic 0 :  becoming,post,must,score,begin,due,panel,van,bill,clash
Topic 1 :  heart,sexually,dev,play,dhoni,country,quick,next,skip,comeback
Topic 2 :  index,paul,converted,fatima,bar,solar,people,game,case,becomes
Topic 3 :  japan,implicate,busted,necessary,airport,world,air,year,deal,sex
Topic 4 :  musk,stone,billie,past,never,trade,early,christmas,analytics,initial
Topic 5 :  data,first,lost,every,amid,exchange,return,choice,sale,ca
Topic 6 :  sanjay,gift,award,poor,jan,elon,industry,working,deadlock,bulb
Topic 7 :  yemen,enhanced,board,role,road,poor,catch,trader,iceland,test
Topic 8 :  man,express,temp,self,safe,matchbox,use,film,batsman,musk

# News Data

In [ ]:
cleaned_news=clean_text(news)
cleaned_news[:10]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


["virender sehwag captaining india hit runs west indies december , , becoming second cricketer sachin tendulkar score double hundred odis. sehwag's () highest individual score captain odi cricket history. sehwag's knock helped india pile highest odi total /.",
 'after actor varun dhawan revealed instagram covid-, troll asked, "do really covid-? or overacting normal cough covid-?" varun responded, "what amazing sense humour have...hope family don’t suffer though need suffer sad excuse humour."',
 'actor fardeen khan said feels like he\'s losing kilograms six months, target feel like he\'s . the -year-old actor told toi, "it year i feeling anymore...i wanted physically feel ." he added, "i started eating right healthy...and coupled correct workouts."',
 't natarajan, made ti debut ongoing series australia, took twitter shared pictures second ti india clinched series. "first series win country\u200b. memorable special," wrote. natarajan took five wickets first two ti matches conceded runs

In [ ]:
hmm_lda(news)

Topic 0 :  unexpected,wade,rahul,senior,quash,church,agreement,corporation,upper,death
Topic 1 :  sanjay,later,cook,third,mistakenly,phase,best,set,donald,testing
Topic 2 :  shooting,residence,worth,amid,executive,posted,senior,old,needs,comedy
Topic 3 :  road,short,tough,director,nuclear,panic,collection,subject,fourth,clique
Topic 4 :  land,top,brought,banking,east,luckily,total,speech,titled,gotten
Topic 5 :  top,given,gal,ed,link,overall,israel,compulsory,sports,receive
Topic 6 :  wilson,loss,anas,record,franchise,pilot,wrote,plain,protest,felt
Topic 7 :  also,much,falling,singer,national,bitten,paul,inaugural,burden,media
Topic 8 :  huge,positive,decided,turki,statement,move,way,company,deliver,man
Topic 9 :  son,nine,striker,ever,upcoming,bagged,order,international,need,group
Topic 10 :  steve,instead,political,medal,fact,war,made,travelled,john,might
Topic 11 :  defamation,reportedly,family,evening,hospital,reception,akali,access,billion,temperature
Topic 12 :  union,ahead,score

#Scientific journals data

In [ ]:
abstracts_cleaned=clean_text((scientific_df["abstarcts"]).tolist())

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
abstracts_cleaned[:10]

['"... in context appearance-based paradigm object recognition, generally believed algorithms based lda (linear discriminant analysis) superior based pca (principal components analysis) . in communication show always case. we present ..."',
 '"... we describe latent dirichlet allocation (lda), generative probabilistic model collections discrete data text corpora. lda three-level hierarchical bayesian model, item collection modeled finite mixture underlying set topics. each topic is, ..."',
 '"... improvement tailoring near-wall turbulence ..."',
 '"... we consider problem modeling annotated data—data multiple types instance one type (such caption) serves description type (such image). we describe three hierarchical probabilistic mixture models aimed data, culminating cor ..."',
 '"... linear discriminant analysis (lda) successfully used dimensionality reduction technique ..."',
 '"... abstract—we propose appearance-based face recognition method called laplacianface approach. by using l

In [ ]:
hmm_lda(abstracts_cleaned)

Topic 0 :  even,show,behaviour,useful,different,risk,physics,full,belief,showing
Topic 1 :  subset,ionization,mixture,structure,vector,unlike,however,technique,place,measure
Topic 2 :  task,walk,many,skew,overview,search,resource,work,segmentation,number
Topic 3 :  linear,maximum,covered,generalized,issue,large,mobile,onset,statistical,operator
Topic 4 :  linear,limited,reliable,al,kind,news,resonance,mental,advertisement,training
Topic 5 :  linear,visual,specific,technique,many,still,set,class,work,large
Topic 6 :  technique,rapidly,unrestricted,grasp,combined,efficacy,categorization,walking,plus,matching
Topic 7 :  broken,successful,many,geometry,class,mode,sample,normal,improving,generate
Topic 8 :  dealing,technical,conventional,elaborate,proven,diagnosis,project,mainly,understanding,component
Topic 9 :  linear,student,fixed,useful,without,firstly,task,known,dimensionality,herbal
Topic 10 :  size,extended,issue,class,present,quotation,many,include,fringe,learning
Topic 11 :  high,a

# Social media data( Twitter data)

In [ ]:
tweets=clean_text(social_media[2].tolist())

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
tweets[:10]

['rt @proudsocialist: joe biden, nancy pelosi, democratic party refusing support medicare all pandemic, gre…',
 'rt @macfarlanenews: new: office special coiunsel finds white house trade advisor peter narvarro violated federal hatch act atta…',
 'rt @cmclymer: congratulations joe biden becoming first presidential candidate win georgia four times one year.',
 "rt @joncoopertweets: michigan gov. gretchen whitmer serve one five co-chairs president-elect joe biden's inauguration, b…",
 'rt @nytimes: breaking news: joe biden plans tap retired gen. lloyd austin defense secretary. he would first african-ame…',
 '@robreiner crystal clear: isn’t asking anyone lie. he simply asking validation. crystal clear: while… https://t.co/eqfjatkgk',
 'abp. aquila backs call pro-abortion ‘catholic’ joe biden denied holy communion | news | lifesite https://t.co/cvuokbvkzh',
 'atlanta — georgia’s top elections official monday recertified state’s election results recount reque… https://t.co/rynoin',
 'rt @fair

In [ ]:
hmm_lda(tweets)

Topic 0 :  lead,year,thank,slipped,gable,one,evidently,funds,broke,pure
Topic 1 :  daily,series,recount,cast,voting,ga,early,house,de,man
Topic 2 :  shooting,presidency,china,help,office,tell,roundly,kids,monday,early
Topic 3 :  austin,already,heavily,certified,utter,finding,meet,china,use,important
Topic 4 :  tide,official,second,morning,chosen,candidate,delaware,poll,record,legally
Topic 5 :  austin,voting,go,corps,plan,summer,force,could,nominate,electoral
Topic 6 :  legal,news,ga,finding,bill,wide,today,electoral,one,lawyer
Topic 7 :  austin,candidate,finding,twitter,us,host,office,mass,times,hear
Topic 8 :  elaborate,pandemic,rudy,oh,county,please,parade,crack,today,emergency
Topic 9 :  four,speaker,time,happy,communism,house,series,funds,per,health
Topic 10 :  praising,discord,voting,donald,popular,bice,refuse,sure,afraid,time
Topic 11 :  swung,trust,speaker,unacceptable,second,general,las,larry,presidente,finale
Topic 12 :  many,wrote,house,general,time,operate,army,becoming,ele

# Legal data

In [ ]:
legal_df=pd.read_csv("/content/legal.csv")

In [ ]:
legal_df[:10]

,bill_name,bill_number,broad_subjects,committee,narrow_subjects,purpose,sponsor,status,url
0,Harford County Sheriff - Deputy Sheriffs and C...,HB0015,Courts and Court Personnel - Local,Appropriations,"Collective Bargaining,Contracts -see also- Lan...",FOR the purpose of providing that certain depu...,Delegate Lisanti,In the House - Unfavorable Report by Appropria...,http://mgaleg.maryland.gov/2018RS/bills/hb/hb0...
1,Creation of a State Debt - Howard County - Har...,HB0061,State Debt (Bond Bills),Appropriations,"Community Facilities and Services,Howard Count...",FOR the purpose of authorizing the creation of...,Delegate Turner,In the House - Unfavorable Report by Appropria...,http://mgaleg.maryland.gov/2018RS/bills/hb/hb0...
2,Creation of a State Debt – Anne Arundel County...,HB0051,State Debt (Bond Bills),Appropriations,"Anne Arundel County,Public Schools,Sports and ...",FOR the purpose of authorizing the creation of...,Delegate Beidle,In the House - Unfavorable Report by Appropria...,http://mgaleg.maryland.gov/2018RS/bills/hb/hb0...
3,Creation of a State Debt - Anne Arundel County...,HB0045,State Debt (Bond Bills),Appropriations,"Anne Arundel County,Public Schools,Sports and ...",FOR the purpose of authorizing the creation of...,Delegate Carey,In the House - Unfavorable Report by Appropria...,http://mgaleg.maryland.gov/2018RS/bills/hb/hb0...
4,Employees' and Teachers' Retirement and Pensio...,"SB1252,(CH0469)",State Pensions and Retirement,"Budget and Taxation,Appropriations",Administrative Agencies -see also- Electronic ...,NaN,Senator Peters,Approved by the Governor - Chapter 469,http://mgaleg.maryland.gov/2018RS/bills/sb/sb1...
5,Corporate Income Tax - Single Sales Factor App...,"HB1794,(CH0342)",Taxes - Income,"Ways and Means,Budget and Taxation",Corporations -see also- Municipal Corps; Publi...,FOR the purpose of altering the formula used t...,Delegate Kaiser,Approved by the Governor - Chapter 342,http://mgaleg.maryland.gov/2018RS/bills/hb/hb1...
6,Charles County - Cigarette Retailers - County ...,SB1266,"Counties - Local Laws,Business Regulation and ...","Education, Health, and Environmental Affairs,E...","Charles County,Cigarettes -see also- Smoking,C...",FOR the purpose of increasing the fee f...,Senator Middleton,In the House - Rereferred to Economic Matters,http://mgaleg.maryland.gov/2018RS/bills/sb/sb1...
7,Offshore Drilling Liability Act,SB1128,Civil Actions and Procedures,"Judicial Proceedings,Judiciary,Environment and...",Bonds -see also- County & Baltimore City Bonds...,FOR the purpose of establishing that an offsho...,Senator Mathias,Vetoed by the Governor (Duplicative),http://mgaleg.maryland.gov/2018RS/bills/sb/sb1...
8,Child Abuse and Neglect - Substance-Exposed Ne...,"HB1744,(CH0410)",Family Law,"Judiciary,Judicial Proceedings","Child Abuse and Neglect,Controlled Dangerous S...",FOR the purpose of altering the conditio...,Delegate Wilson,Approved by the Governor - Chapter 410,http://mgaleg.maryland.gov/2018RS/bills/hb/hb1...
9,Education – Identification of Students With Tr...,HB1530,"Primary and Secondary Education,Public Health","Ways and Means,Rules",Administrative Agencies -see also- Electronic ...,FOR the purpose of requiring county boards of ...,Delegate Flanagan,In the Senate - First Reading Senate Rules,http://mgaleg.maryland.gov/2018RS/bills/hb/hb1...


In [ ]:
legal_data=clean_text(legal_df['purpose'].dropna().tolist())

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
hmm_lda(legal_data)

Topic 0 :  actual,form,surviving,revoke,minimum,control,neurological,endowment,patient,prohibition
Topic 1 :  industrial,benefit,common,design,physical,living,address,uninsured,district,parent
Topic 2 :  area,pipe,pending,zone,percentage,solicit,river,meet,university,organization
Topic 3 :  deed,safety,youth,revert,high,overhead,ultimately,intentionally,finding,pornography
Topic 4 :  abandoned,district,vacate,course,address,ecumenical,opening,authorize,appoint,missing
Topic 5 :  annually,misconduct,related,level,contain,washington,industrial,damages,affect,crisis
Topic 6 :  spouse,appointment,determination,congressional,proceeding,ordinary,available,foundation,standard,inspector
Topic 7 :  platform,supplement,technology,course,gather,ad,liquor,peaceful,aggregate,month
Topic 8 :  loss,thoroughbred,supplemental,certified,seek,another,continue,wind,reasonable,link
Topic 9 :  appropriate,stylistic,voting,respectively,available,late,increasing,administrative,fill,cash
Topic 10 :  campaign,f